# Creating hardware in Python using CyriteHDL

CyriteHDL or short CyHDL is a **hardware definition toolkit** based on a kernel using an internal representation python notation to generate hardware or code procedurally. It is featured by an emulation layer to provide some (not full) language compatibility to the MyHDL dialect, with the following major differences:

* Strictly typed interfaces
* Signal extensions to elleviate complex bus system designs
* Modular output to VHDL and Verilog
* Support for blackboxes, builtins and libraries
* Direct synthesis via yosys, fast CXXRTL simulations

Moreover, it is possible to write closed source code generators using an augmented MyHDL syntax. Note though that Cyrite enforces some stricter design rules and some constructs are no longer supported.

If you are aiming at a fresh start with a Python based HDL framework, this howto should get you going with the following basics:

1. The (in)famous [blinky](blinky.ipynb)
   1. Designing a counter, state machine and configuration register
   1. Testing and simulating its function
2. Synthesizing a design for a FPGA target
   1. Creating the board supply package
   1. Direct synthesis to yosys
   1. Synthesis via transfer language (Verilog, VHDL)

## Short peek into internals

The cyrite library is based on a generator kernel that creates the resulting hardware or language target elements by execution rather than AST translation. This internal representation layer is referred to as the myirl kernel. All cyrite or MyHDL alike hardware descriptions on top however are AST-translated into a generator form. An example:

In [1]:
import sys
sys.path.insert(0, "../../")

### A simple multiplexer unit

Normally, you would write a behavioral model for a multiplexer as follows:

In [2]:
from cyhdl import *

@block
def mux_unit(en: Signal, a_in : Signal, b_out : Signal):
    @always(en, a_in)
    def muxer():
        if en:
            b_out.next = a_in
        else:
            b_out.next = ~a_in
    return muxer

    @sequence
    def main():
        a.next = 0

Simple top level `@block` can be displayed in their explicit IRL:

In [3]:
print(mux_unit.unparse())

Unparsing unit mux_unit


@block
def mux_unit(en: Signal, a_in: Signal, b_out: Signal):

    @always_(en, a_in)
    def muxer(_context):
        (yield [_context.If(en).Then(b_out.set(a_in)).Else(b_out.set((~ a_in)))])
    return muxer

    @sequential
    def main(_sequence):
        _sequence += [a.set(0)]



To display the resulting HDL, we instance a DummyTargetModule design context. When emitting instances within that context, HDL results are sent to sys.stdout. This allows us to examine language constructs in a granular way.

In [4]:
en = Signal(bool())
s = [ Signal(intbv()[4:]) for _ in range(2) ]

from myirl.targets.dummy import DummyTargetModule
ctx = DummyTargetModule(targets.Verilog)

uut = mux_unit(en, s[0], s[1])

for inst in uut.instances:
    inst(ctx) # Call instances to within context
    inst.emit(ctx)


always @ (en or a_in) begin : MUXER
    if (en) begin
        b_out <= a_in; /* fallback */
    end else begin
        b_out <= ~a_in; /* fallback */
    end
end


Note: at this stage, no distinction between Verilog type reg and wire happens, because no signal analysis has been done. Therefore a 'fallback' tag comment is inserted by the translation.

## The Cyrite API reference

**Under construction. Please refer to the MyIRL internals for the time being**

One important note: The Cyrite HDL is split in several layers:

* Cyrite library layer: top level HDL syntax, MyHDL alike
* myirl 'intermediate representation layer' kernel layer: Procedural, generator style syntax and library
  

When dealing with development of new extensions, one is mostly operating in the `myirl` domain. Thus, `myirl.library` elements mostly use IRL (intermediate representation layer) notation.
Some elements are only documented in the IRL domain, however can be used from the cyrite layer. This is yet not fully coverage-tested.

* [Generator basics](generators.ipynb) - When to use `yield` and when not
* [Signal types and Interfaces](signals_interfaces.ipynb)
  * [Port classes](ports.ipynb)
  * [Tristate and undefined signals](stdlogic.ipynb)
* [Library concepts: Organizing your code](library.ipynb)
    * [Blackboxes, external HDL modules](library.ipynb#External-HDL-libraries)
    * Inference and design modules
* Cyrite/MyIRL Library and extensions
    * [MyIRL library reference](../../myirl/library/index.ipynb)
    * [Register concepts](../notebooks/registers.ipynb)
         * Partially assigneable signals [[myirl kernel ref]](../notebooks/member_assignment.ipynb)
         * Address decoders, [Busses](../notebooks/busses.ipynb)
         * Memory mapped range registers
* Synthesis versus simulation
    * [Simulation examples](simulation.ipynb) and scenarios, waveform displays
    * [Simulation constructs](../notebooks/simulation.ipynb)
       * [Timing delays, delayed signals](../notebooks/simulation.ipynb#Delay-modelling)
    * [Co-Simulation](../notebooks/simulation.ipynb#Co-Simulation-abstraction): Driving virtual hardware via CXXRTL
    * [Simulation commands reference](../notebooks/simulation.ipynb#Simulation-command-reference)